In [1]:
import pandas as pd
import numpy as np
import pymssql
import pyodbc
from datetime import datetime
import win32com
import win32com.client
import re
import threading
import time

In [16]:
class Email_Utils():
    def __init__(self):
        super().__init__()
        self.outlook = win32com.client.Dispatch("Outlook.Application")
        self.Rxoutlook = self.outlook.GetNamespace("MAPI")
        self.mail_receivers = "dany.shin@hanwha.com"

    def send_email(self, subject, body):    
        Txoutlook = outlook.CreateItem(0)
        Txoutlook.To = self.mail_receivers
        Txoutlook.Subject = subject
        Txoutlook.HTMLBody = f"""{body}"""
        Txoutlook.send()    

    def sender_mailaddr_extract(self, i):
        if i.SenderEmailType=='EX':
            if i.Sender.GetExchangeUser() != None:
                return i.Sender.GetExchangeUser().PrimarySmtpAddress
            else:
                return msg.Sender.GetExchangeDistributionList().PrimarySmtpAddress
        else:
            return i.SenderEmailAddress

    def recevie_email(self, check_date, download_filetype, saveYN, save_path):
        recip = self.Rxoutlook.CreateRecipient(self.mail_receivers)
        inbox = self.Rxoutlook.GetSharedDefaultFolder(recip, 6)
        for i in inbox.items:
            atts = [att for att in i.Attachments if download_filetype in att.FileName.split('.')[-1]]
            if len(atts) > 0:
                if datetime.strptime(i.SentOn.strftime('%Y-%m-%d'), '%Y-%m-%d') >= datetime.strptime(check_date, '%Y-%m-%d'):
                    print('=' * 10, i.SentOn.strftime('%Y-%m-%d'), '=' * 10)
                    print(i.subject) # 메일제목
                    try:
                        print(i.Sender, self.sender_mailaddr_extract(i), i.CC) #메일 발신인
                    except:
                        pass

                    for att in atts:
                        print(att.FileName) #메일 첨부파일    
                        if saveYN == True:
                            att.SaveAsFile(save_path + '/{}'.format(att.FileName))
                    print('\n')
                
    def main(self):
        if (datetime.now().second == 1):
            email_rpa.recevie_email(check_date = 'abc', 
                        download_filetype = 'xlsx', 
                        saveYN = False,
                        save_path = './data')
        else:
            print('waiting for next batch')
        threading.Timer(1, self.main).start() 


In [24]:
email_rpa = Email_Utils()
email_rpa.recevie_email(check_date = '2022-03-28', 
            download_filetype = 'xlsx', 
            saveYN = False,
            save_path = './data')

# email_rpa.main()

========== 2022-03-28 ==========
2022년 03월 27일  2동 기자재창고 Spare Part’s 일일 재고현황 공유 件
rm.jc2 rm.jc2@qcells.com
이병근(byoungkeun.lee); 김기영(kiyoung.kim); 성기석(GiSeok.Seong); 정한준(hanjun.jung); 염규현(Kyuhyun.Yeom); 김성진(Seongjin.Kim); rm.jc1; 김대호(Daeho.Kim); 고성민(Sungmin.Ko); 전미선(Miseon Jeon); 김근우(KeunWoo.Kim); 정동순(Dongsoon.Jung); 조하식(hasik.cho); 노경미(Kyoungmi.No); 김범수(Bumsoo.Kim); 허문영(MOONYOUNG HEO); 백충식(CHOONGSIK.BAEK); 윤성규(sungkyu.yoon); 이준배(Junbae Lee); 서윤석(Yoonseok.Seo); 김상우(Sangwoo1.Kim); utmp; 이정규(JUNGGYEU LEE); 김승용; 이재후(JAE HU LEE); 김동주(Dongju.Kim); 남상기(Sangki.Nam); 이경수(Gyeongsu.Lee); 오태준(Taejun.Oh); 유현민(Hyunmin.Yu); 정흥렬(Heungryeol.Jeong); 김찬중(Chanjung.Kim); 이세원(Sewon Lee); 구연하(yeonha.koo); 정인석(Inseok.Jung); 김철민(Chulmin.Kim); 지대건(Daegeon.Ji); 이명재(Myoungjae.Lee); 권기봉(Kibong.Kwon); 박상철(Sangchul.Park); 이규은(Kyueun.Lee); 김성진(Sungjin1.Kim); 오찬현(Chanhyun.Oh); 최상진(sangjin.choi); 신경섭(Kyoungsup Shin); 이경우(Kyoungwoo.Lee); 경명진(myungjin.kyung); 김삼일(Samil.Kim); 김동준(Dongjun.Kim); 송명근(Myeonggeun.Song); 왕승업(S

In [ ]:
class DB_Utils(Email_Utils):
    def connect_mssql(self, server, port, username, password, database):   
        self.cnxn =  pymssql.connect(server, port, username, password, database)
        self.cursor = self.cnxn.cursor()
        log.Log_Write('Q.BIZ DB접속 완료')

    def connect_azuredb(self):
        server = '<server>.database.windows.net'
        database = '<database>'
        username = '<username>'
        password = '{<password>}'   
        driver= '{ODBC Driver 17 for SQL Server}'        
        conn = pyodbc.connect('DRIVER='+driver+
                              ';SERVER=tcp:'+server+
                              ';PORT=1433;DATABASE='+database+
                              ';UID='+username+
                              ';PWD='+ password)
    def fetch_data(self, sql):
        self.cursor.execute(sql)
        row = self.cursor.fetchall()
        col_names = [item[0] for item in self.cursor.description]
        return pd.DataFrame(row, columns=col_names)

    def close_db(self):
        self.cnxn.close()
        log.Log_Write('Q.BIZ DB접속 종료')

    def delete_data(self, sql):
        self.cursor.execute(sql)
        self.cnxn.commit()
        log.Log_Write('Q.BIZ DB 데이터 삭제 완료')

    def insert_data(self, sql, df):
        for row in range(len(df)):
            time.sleep(0.02)
            print(df.values[row])
            self.cursor.execute(sql, tuple(df.values[row]))
        self.cnxn.commit()
        log.Log_Write('Q.BIZ DB 데이터 입력 완료')


In [ ]:
class ETL_Utils(DB_Utils):
    def vartify_attachment_names(self):
        #read_metatable
        #text ssim
        #check threshold
        pass
    
    def meta_sheet(self, excel_name):
        sql = f'''select sheet_list, primary_key, target_columns from meta_sheet_info
                where file_name = {excel_name}'''
        self.sheet_info = self.fetch_data(sql)

    def clean_data(self):
        '''
            dropnan -> select target columns
        '''
        pk = self.sheet_info[self.sheet_info == sheet]['primary_key'].values[0]
        target_cols = self.sheet_info[self.sheet_info == sheet]['target_columns'].tolist()
        
        df = df.dropna(subset = pk).reset_index(drop = True)
        df = df[target_cols]
        return df

    def read_excel(self, path):
        '''
            Read excel file -> drop unnamed cols -> select target sheets as dataframe
        '''
        try:
            if path[-4:] == 'xlsb':
                df_obj = pd.ExcelFile(path,engine='pyxlsb')
            else:
                df_obj = pd.ExcelFile(path,engine='openpyxl')

            self.df_list = []
            self.meta_sheet(path)

            for sheet in self.sheet_info['sheet_list'].tolist():
                df = df_obj.parse(sheet_name=sheet)
                df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
                df = self.clean_data(df)
                self.df_list.append(df)
        except Exception as e:
            print(e)
            self.send_email(path, e)
            pass
        


In [ ]:
class Report_Log():
    log = None
    dir_name = ''
    file_name = ''
    
    def __init__(self, file_name, propagate = True):
        self.file_name = file_name
        Report_Log.log = None
        Report_Log.dir_name = os.path.dirname(os.path.abspath(__file__)) + '/log/'   
        self.propagate = propagate
        Report_Log.Create_Log(self)
              
    def Create_Log(self) :
        file_max_byte = 1024 * 1024 * 10
        log_name = Report_Log.dir_name + self.file_name + '.log'
        print(log_name)

        Report_Log.log = logging.getLogger('Report_Log')
        Report_Log.log.setLevel(logging.DEBUG)

        formatter = logging.Formatter('[%(asctime)s] [%(levelname)s]  %(message)s')

        fileHandler = logging.handlers.RotatingFileHandler(log_name, maxBytes=file_max_byte, backupCount=10,  encoding='utf-8')
        streamHandler = logging.StreamHandler()

        fileHandler.setFormatter(formatter)
        streamHandler.setFormatter(formatter)
        if self.propagate == True:
            Report_Log.log.addHandler(fileHandler)
            Report_Log.log.addHandler(streamHandler)
        
    def Log_Write(self, msg, level='debug') :
        if level == 'debug' :
            Report_Log.log.debug(msg)
        elif level == 'info' :
            Report_Log.log.info(msg)
        elif level == 'warning' :
            Report_Log.log.warning(msg)
        elif level == 'error' :
            Report_Log.log.error(msg)
        elif level == 'critical' :
            Report_Log.log.critical(msg)
        else :
            Report_Log.log.debug(msg)